# Delta Lake Testing

Testing out the abilities of deltalake

In [1]:
import os

In [2]:
packages = "io.delta:delta-core_2.12:1.0.0,org.apache.hadoop:hadoop-aws:3.2.0"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages {0} pyspark-shell".format(packages)

In [3]:
from pyspark.sql import SparkSession

In [4]:
# .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
spark = SparkSession \
    .builder \
    .appName("Load Data") \
    .config("spark.master", "spark://spark-master:7077") \
    .config("spark.num.executors", "2") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .enableHiveSupport() \
    .getOrCreate()

## Loading in some data

In [5]:
green_trip_data_2015_test = "s3a://nyc-tlc/trip data/green_tripdata_2015-01.csv"
green_trip_2015_test = spark.read.option("header", True).csv(green_trip_data_2015_test)

In [13]:
green_trip_2015_test = green_trip_2015_test.withColumnRenamed("Trip_type ", "trip_type")

In [14]:
green_trip_2015_test.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- lpep_pickup_datetime: string (nullable = true)
 |-- Lpep_dropoff_datetime: string (nullable = true)
 |-- Store_and_fwd_flag: string (nullable = true)
 |-- RateCodeID: string (nullable = true)
 |-- Pickup_longitude: string (nullable = true)
 |-- Pickup_latitude: string (nullable = true)
 |-- Dropoff_longitude: string (nullable = true)
 |-- Dropoff_latitude: string (nullable = true)
 |-- Passenger_count: string (nullable = true)
 |-- Trip_distance: string (nullable = true)
 |-- Fare_amount: string (nullable = true)
 |-- Extra: string (nullable = true)
 |-- MTA_tax: string (nullable = true)
 |-- Tip_amount: string (nullable = true)
 |-- Tolls_amount: string (nullable = true)
 |-- Ehail_fee: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- Total_amount: string (nullable = true)
 |-- Payment_type: string (nullable = true)
 |-- trip_type: string (nullable = true)



In [15]:
green_trip_2015_test.write.format("delta").save("/opt/spark-data/temp/taxi_test")

# Try a bigger load with schema evolution

In [23]:
table_path = "/opt/spark-data/temp/green_2015"

In [12]:
green_trip_data_2015_h1 = "s3a://nyc-tlc/trip data/green_tripdata_2015-0[1-6].csv"
green_trip_2015_h1 = spark.read.option("header", True).csv(green_trip_data_2015_h1)

In [13]:
green_trip_2015_h1.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- lpep_pickup_datetime: string (nullable = true)
 |-- Lpep_dropoff_datetime: string (nullable = true)
 |-- Store_and_fwd_flag: string (nullable = true)
 |-- RateCodeID: string (nullable = true)
 |-- Pickup_longitude: string (nullable = true)
 |-- Pickup_latitude: string (nullable = true)
 |-- Dropoff_longitude: string (nullable = true)
 |-- Dropoff_latitude: string (nullable = true)
 |-- Passenger_count: string (nullable = true)
 |-- Trip_distance: string (nullable = true)
 |-- Fare_amount: string (nullable = true)
 |-- Extra: string (nullable = true)
 |-- MTA_tax: string (nullable = true)
 |-- Tip_amount: string (nullable = true)
 |-- Tolls_amount: string (nullable = true)
 |-- Ehail_fee: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- Total_amount: string (nullable = true)
 |-- Payment_type: string (nullable = true)
 |-- Trip_type : string (nullable = true)



In [14]:
green_trip_2015_h1 = green_trip_2015_h1.withColumnRenamed("Trip_type ", "trip_type")

In [20]:
green_trip_2015_h1.write.format("delta").save(table_path)

## With Schema Evolution

In [21]:
green_trip_data_2015_jul_sept = "s3a://nyc-tlc/trip data/green_tripdata_2015-0[7-9].csv"
green_trip_2015_jul = spark.read.option("header", True).csv(green_trip_data_2015_jul_sept)

In [22]:
green_trip_2015_jul = green_trip_2015_jul.withColumnRenamed("Trip_type ", "trip_type")

In [24]:
green_trip_2015_jul.write.format("delta").mode("append").save(table_path)

In [25]:
green_trip_2015_jul.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- lpep_pickup_datetime: string (nullable = true)
 |-- Lpep_dropoff_datetime: string (nullable = true)
 |-- Store_and_fwd_flag: string (nullable = true)
 |-- RateCodeID: string (nullable = true)
 |-- Pickup_longitude: string (nullable = true)
 |-- Pickup_latitude: string (nullable = true)
 |-- Dropoff_longitude: string (nullable = true)
 |-- Dropoff_latitude: string (nullable = true)
 |-- Passenger_count: string (nullable = true)
 |-- Trip_distance: string (nullable = true)
 |-- Fare_amount: string (nullable = true)
 |-- Extra: string (nullable = true)
 |-- MTA_tax: string (nullable = true)
 |-- Tip_amount: string (nullable = true)
 |-- Tolls_amount: string (nullable = true)
 |-- Ehail_fee: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- Total_amount: string (nullable = true)
 |-- Payment_type: string (nullable = true)
 |-- trip_type: string (nullable = true)



In [7]:
green_trip_data_pre2015 = "s3a://nyc-tlc/trip data/green_tripdata_201[3-4]*.csv"
green_trip_pre2015 = spark.read.option("header", True).csv(green_trip_data_pre2015)

In [8]:
green_trip_pre2015.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- lpep_pickup_datetime: string (nullable = true)
 |-- Lpep_dropoff_datetime: string (nullable = true)
 |-- Store_and_fwd_flag: string (nullable = true)
 |-- RateCodeID: string (nullable = true)
 |-- Pickup_longitude: string (nullable = true)
 |-- Pickup_latitude: string (nullable = true)
 |-- Dropoff_longitude: string (nullable = true)
 |-- Dropoff_latitude: string (nullable = true)
 |-- Passenger_count: string (nullable = true)
 |-- Trip_distance: string (nullable = true)
 |-- Fare_amount: string (nullable = true)
 |-- Extra: string (nullable = true)
 |-- MTA_tax: string (nullable = true)
 |-- Tip_amount: string (nullable = true)
 |-- Tolls_amount: string (nullable = true)
 |-- Ehail_fee: string (nullable = true)
 |-- Total_amount: string (nullable = true)
 |-- Payment_type: string (nullable = true)
 |-- Trip_type : string (nullable = true)



In [10]:
green_trip_pre2015 = green_trip_pre2015.withColumnRenamed("Trip_type ", "trip_type")

In [31]:
green_trip_pre2015.write.format("delta").mode("append").save(table_path)

So append with missing column is fine

redo so that we add new column 

In [5]:
path_2 = "/opt/spark-data/temp/green_2015_test"

In [11]:
green_trip_pre2015.write.format("delta").mode("append").save(path_2)

In [15]:
green_trip_2015_h1.write.format("delta").mode("append").save(path_2)

AnalysisException: A schema mismatch detected when writing to the Delta table (Table ID: 1c46d9f3-4b5b-4034-80dc-46f43d7264de).
To enable schema migration using DataFrameWriter or DataStreamWriter, please set:
'.option("mergeSchema", "true")'.
For other operations, set the session configuration
spark.databricks.delta.schema.autoMerge.enabled to "true". See the documentation
specific to the operation for details.

Table schema:
root
-- VendorID: string (nullable = true)
-- lpep_pickup_datetime: string (nullable = true)
-- Lpep_dropoff_datetime: string (nullable = true)
-- Store_and_fwd_flag: string (nullable = true)
-- RateCodeID: string (nullable = true)
-- Pickup_longitude: string (nullable = true)
-- Pickup_latitude: string (nullable = true)
-- Dropoff_longitude: string (nullable = true)
-- Dropoff_latitude: string (nullable = true)
-- Passenger_count: string (nullable = true)
-- Trip_distance: string (nullable = true)
-- Fare_amount: string (nullable = true)
-- Extra: string (nullab

In [16]:
green_trip_2015_h1.write.format("delta").mode("append").option("mergeSchema", "true").save(path_2)

In [34]:
spark.stop()